In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Get the project root directory (sciborg root)
current_dir = Path.cwd()
project_root = current_dir

# Look for sciborg root by going up until we find .env or sciborg directory
while project_root != project_root.parent:
    if (project_root / '.env').exists() or project_root.name == 'sciborg':
        break
    project_root = project_root.parent
    if project_root == project_root.parent:  # Reached filesystem root
        # Fallback: assume we're in notebooks/testing, go up two levels
        project_root = Path.cwd()
        if 'notebooks' in str(project_root):
            project_root = project_root.parent.parent
        break

# Load .env file from project root
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(dotenv_path=str(env_path))
    print(f"Loaded .env from: {env_path}")
else:
    # Try loading from current directory as fallback
    load_dotenv()
    print("Warning: .env not found in project root, trying current directory")

# Get API keys from environment
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key:
    os.environ["OPENAI_API_KEY"] = openai_key
else:
    print("Warning: OPENAI_API_KEY not found in environment variables")



In [2]:
import os
import sys

sys.path.insert(0, str(project_root)) 

from langchain_openai import ChatOpenAI
from sciborg.ai.agents.core import create_linqx_chat_agent
from sciborg.ai.agents.core2 import SciborgAgent
from sciborg.ai.chains.microservice import module_to_microservice
from sciborg.ai.chains.workflow import create_workflow_planner_chain, create_workflow_constructor_chain

from sciborg.utils.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from sciborg.core.library.base import BaseDriverMicroservice


In [3]:
file_path = 'str(project_root / 'ai/agents/driver_pubchem.json'

pubchem_command_microservice = module_to_microservice(PubChemCaller)
print(pubchem_command_microservice)

with open(file_path, 'w') as outfile:
    outfile.write(pubchem_command_microservice.model_dump_json(indent=2))


Processing functions:  12%|█▎        | 1/8 [00:02<00:19,  2.77s/function]

{'name': 'get_assay_description_from_AID', 'microservice': 'PubChemCaller', 'desc': 'Function purpose Get high level Assay description, protocol and comment on the scores for a given assay id.', 'uuid': '6c8cce34-c263-4274-a918-b6d05d0701f2', 'parameters': {'aid': {'name': 'aid', 'data_type': 'str', 'precision': -1, 'upper_limit': None, 'lower_limit': None, 'allowed_values': [], 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'var_name': '', 'desc': 'string representation of a list of assay IDs'}}, 'has_return': True, 'return_signature': {'dict': 'a dictionary of description, protocol and comment for the given assay ID'}}
<class 'dict'>


Processing functions:  25%|██▌       | 2/8 [00:04<00:12,  2.08s/function]

{'name': 'get_assay_name_from_aid', 'microservice': 'PubChemCaller', 'desc': 'Function purpose\nGives a dictionary of names for each assay ID (aid)', 'uuid': '6c8cce34-c263-4274-a918-b6d05d0701f2', 'parameters': {'aid': {'name': 'aid', 'data_type': 'str', 'desc': 'string representation of a list of assay IDs', 'is_optional': False, 'is_list': False, 'from_var': False}}, 'has_return': True, 'return_signature': {'str': 'a dictionary of names for each assay ID'}}
<class 'dict'>


Processing functions:  38%|███▊      | 3/8 [00:06<00:09,  1.95s/function]

...

{'name': 'get_description', 'microservice': 'PubChemCaller', 'desc': "Function purpose\nGet description of a substance or a compound, for assay description, use get_assay_description() instead\n\nInputs\ninp: string representation of an single chemical name only\ninp_format: string of either of name, sid, cid, smiles \ninp_type: 'compound' if inp_format is cid, name or smiles of compound or 'substance' if inp_format is sid, name, smiles of substance\n\nreturns\ndict: a dictionary of descriptions for each identifier", 'uuid': '6c8cce34-c263-4274-a918-b6d05d0701f2', 'parameters': {'inp': {'name': 'inp', 'data_type': 'str', 'desc': 'string representation of an single chemical name only', 'is_optional': False, 'is_list': False, 'from_var': False}, 'inp_format': {'name': 'inp_format', 'data_type': 'str', 'desc': 'string of either of name, sid, cid, smiles', 'is_optional': False, 'is_list': False, 'from_var': False}, 'inp_type': {'name': 'inp_type', 'data_type': 'str', 'desc': "'compound' if

Processing functions:  88%|████████▊ | 7/8 [00:19<00:03,  3.11s/function]

{'name': 'get_smiles_from_name', 'microservice': 'PubChemCaller', 'desc': 'Function purpose\nGet SMILES for a given chemical name.', 'uuid': '6c8cce34-c263-4274-a918-b6d05d0701f2', 'parameters': {'inp': {'name': 'inp', 'data_type': 'str', 'desc': 'string representation of a chemical name', 'is_optional': False, 'is_list': False, 'from_var': False, 'default': None}}, 'has_return': True, 'return_signature': {'str': 'a text representing the SMILES for the given chemical name'}}
<class 'dict'>


Processing functions: 100%|██████████| 8/8 [00:21<00:00,  2.72s/function]

{'name': 'get_synonym', 'microservice': 'PubChemCaller', 'desc': 'Function purpose Get Synonym of a substance or compound.', 'uuid': '6c8cce34-c263-4274-a918-b6d05d0701f2', 'parameters': {'inp': {'name': 'inp', 'data_type': 'str', 'desc': 'string representation of a list of identifiers', 'is_list': False, 'is_optional': False}, 'inp_format': {'name': 'inp_format', 'data_type': 'str', 'desc': 'string which can be either of name, sid, cid, smiles', 'is_list': False, 'is_optional': False}, 'inp_type': {'name': 'inp_type', 'data_type': 'str', 'desc': "'compound' if inp_format is cid, name or smiles of compound or 'substance' if inp_format is sid, name, smiles of substance", 'is_list': False, 'is_optional': False}}, 'has_return': True, 'return_signature': {'dict': 'a dictionary of synonyms for each identifier'}}
<class 'dict'>
name='PubChemCaller' uuid=UUID('6c8cce34-c263-4274-a918-b6d05d0701f2') desc='This is the Module for Pubchem API caller.\n\nThis is one of the best Chemical databases.

In [5]:
file_path = 'str(project_root / 'ai/agents/driver_MicroSynth.json'

driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
print(type(driver_command_microservice))

with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))


Processing functions:  12%|█▎        | 1/8 [00:02<00:20,  2.89s/function]

{'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '74a9cea3-1787-40ed-bb63-61d559d95328', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}
<class 'dict'>


Processing functions:  25%|██▌       | 2/8 [00:04<00:12,  2.08s/function]

{'name': 'close_lid', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Closes the lid on the microwave synthesizer. Must be run prior to running heating.', 'uuid': '74a9cea3-1787-40ed-bb63-61d559d95328', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'desc': 'the id of the current session', 'is_optional': False, 'is_list': False, 'from_var': False}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions:  38%|███▊      | 3/8 [00:06<00:10,  2.15s/function]

...

{'name': 'open_lid', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Opens the lid on the microwave synthesizer. Must be run prior to loading a vial.', 'uuid': '74a9cea3-1787-40ed-bb63-61d559d95328', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'desc': 'the id of the current session', 'is_optional': False, 'is_list': False, 'from_var': False}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions:  88%|████████▊ | 7/8 [00:14<00:01,  1.89s/function]

{'name': 'unload_vial', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Unloads a vial from the microwave synthesizer. Must be run after heating.', 'uuid': '74a9cea3-1787-40ed-bb63-61d559d95328', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'desc': 'the id of the current session', 'is_optional': False, 'is_list': False, 'from_var': False}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions: 100%|██████████| 8/8 [00:17<00:00,  2.23s/function]

{'name': 'update_heating_parameters', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Sets the heating parameters of the microwave synthesizer. Must be run prior to heating.', 'uuid': '74a9cea3-1787-40ed-bb63-61d559d95328', 'parameters': {'duration': {'name': 'duration', 'data_type': 'int', 'precision': {'const': -1}, 'upper_limit': 60, 'lower_limit': 1, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A short description of the parameter'}, 'temperature': {'name': 'temperature', 'data_type': 'int', 'precision': {'const': -1}, 'upper_limit': 250, 'lower_limit': 25, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A short description of the parameter'}, 'pressure': {'name': 'pressure', 'data_type': 'float', 'precision': {'const': -1}, 'upper_limit': 10, 'lower_limit': 1, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A short description of the parameter'}, 'session_ID': {'name'

In [4]:
from sciborg.ai.agents.core2 import SciborgAgent
agent_2 = SciborgAgent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    memory_file='core_memory.json',
    # rag_vectordb_path = 'faiss_index/',
    use_memory = ['chat', 'action', 'embedding'],
    return_intermediate_steps=True,
    verbose=True
)

/home/parikh92/sciborg/ai/agents/core2.py:121: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memories.append(ConversationBufferWindowMemory(


In [8]:
for memory in agent_2.memory.memories:
    print(type(memory))
    print(memory)
    print("\n***********\n")

<class 'langchain.memory.buffer_window.ConversationBufferWindowMemory'>
chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='How do you manual tune a SCIEX 4000 QTRAP?', additional_kwargs={}, response_metadata={}), AIMessage(content='The manual tuning procedure for a SCIEX 4000 QTRAP mass spectrometer involves the following steps:\n\n1. Mass range of the 4000 Qtrap is 3-2800 Da.\n2. Enable the MS hardware profile with ESI source.\n3. Connect a standalone syringe pump running at 10 uL/min to the source using peek tubing.\n4. Open a prebuilt method in Analyst labeled Q1.\n5. Under continuous infusion, scan the entire mass range and collect 10 scans on average.\n6. Right-click in TIC to open the mass window.\n7. Click on "replace" or "update" icon to adjust the tune parameters.\n\n(Source: papers/SOP/2024_4000 User Guide.pdf, page 1)', additional_kwargs={}, response_metadata={}), HumanMessage(content='I was optimizing a method for fatty acids on the SCIEX 4000 QTRAP and 

In [9]:
agent_2.invoke(
    {
        "input": "Someone said they already optimized a method for fatty acids, do you know what paramters they used?"
    }
)



> Entering new AgentExecutor chain...
Action:
```
{
    "action": "Final Answer",
    "action_input": "The optimized method for fatty acids on the SCIEX 4000 QTRAP provided by a previous user involved the following parameters: voltage = 25 eV, inlet temperature = 200 Celsius, inlet gas = 15, and nanomate tip voltage = 1600 eV."
}
```


/home/parikh92/sciborg/ai/memory/internal_logging.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt) #this will be deprecated soon



> Finished chain.


{'input': 'Someone said they already optimized a method for fatty acids, do you know what paramters they used?',
 'chat_history': 'Human: How do you manual tune a SCIEX 4000 QTRAP?\nAI: The manual tuning procedure for a SCIEX 4000 QTRAP mass spectrometer involves the following steps:\n\n1. Mass range of the 4000 Qtrap is 3-2800 Da.\n2. Enable the MS hardware profile with ESI source.\n3. Connect a standalone syringe pump running at 10 uL/min to the source using peek tubing.\n4. Open a prebuilt method in Analyst labeled Q1.\n5. Under continuous infusion, scan the entire mass range and collect 10 scans on average.\n6. Right-click in TIC to open the mass window.\n7. Click on "replace" or "update" icon to adjust the tune parameters.\n\n(Source: papers/SOP/2024_4000 User Guide.pdf, page 1)\nHuman: I was optimizing a method for fatty acids on the SCIEX 4000 QTRAP and i found the best parameters were voltage = 25 eV, inlet temperature = 200 celsius, inlet gas = 15, and nanomate tip voltage = 1

In [10]:
agent_2.invoke(
    {
        "input": "Can you list the path you took till now to acquire the information that you have now?"
    }
)



> Entering new AgentExecutor chain...
Action:
```
{
    "action": "Final Answer",
    "action_input": "The information that I have now was acquired through the following path:

1. I called the RAG agent to ask 'What is the manual tuning procedure for a SCIEX 4000 QTRAP mass spectrometer?'.
2. The RAG agent provided the manual tuning procedure.
3. The human shared their optimized parameters for fatty acids on the SCIEX 4000 QTRAP.
4. I verified these parameters twice when the human asked about them.

Please note that the optimized parameters were provided by a human user and not acquired through a database or tool."
}
```

> Finished chain.


{'input': 'Can you list the path you took till now to acquire the information that you have now?',
 'chat_history': 'Human: How do you manual tune a SCIEX 4000 QTRAP?\nAI: The manual tuning procedure for a SCIEX 4000 QTRAP mass spectrometer involves the following steps:\n\n1. Mass range of the 4000 Qtrap is 3-2800 Da.\n2. Enable the MS hardware profile with ESI source.\n3. Connect a standalone syringe pump running at 10 uL/min to the source using peek tubing.\n4. Open a prebuilt method in Analyst labeled Q1.\n5. Under continuous infusion, scan the entire mass range and collect 10 scans on average.\n6. Right-click in TIC to open the mass window.\n7. Click on "replace" or "update" icon to adjust the tune parameters.\n\n(Source: papers/SOP/2024_4000 User Guide.pdf, page 1)\nHuman: I was optimizing a method for fatty acids on the SCIEX 4000 QTRAP and i found the best parameters were voltage = 25 eV, inlet temperature = 200 celsius, inlet gas = 15, and nanomate tip voltage = 1600 eV\nAI: Th

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Literal

class MicrowaveSynthesizerFSA(BaseModel):
    sessionID: str | None = Field(defaullt=None, description='ID of the session allocation or None if no session allocated. Changed by allocating the session')
    lid_status: Literal['open', 'closed'] = Field(default='closed', description='status of the lid. Changed by opening or closing the lid')
    vial_status: Literal['loaded', 'unloaded'] = Field(default='unloaded', description='status of the vial. Changed by loading/unloading the vial')
    vial: str | None = Field(default=None, description='Identifier of the vial loaded, None if no vial is loaded. Changed by loading/unloading the vial')
    heating_status: Literal['not_heating', 'heating'] = Field(default='not_heating', description='Status of heating. Only changed by heating the vial, not by updating the parameters')
    temp: int | None = Field(default=None, description='set temperature to heat at, None if not currently set. Changed by updating the heating parameters')
    duration: int | None = Field(default=None, description='set duration to heat for, None if not currently set. Changed by updating the heating parameters')
    pressure: float | None = Field(default=None, description='set pressure to heat at, None if not currently set. Changed by updating the heating parameters')

ms_agent_chat_fsa_memory = SciborgAgent(
    microservice=driver_command_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='all',
    human_interaction=True,
    verbose=True,
    agent_as_a_fsa=True,
    fsa_object=MicrowaveSynthesizerFSA
)

In [4]:
new_agent = SciborgAgent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    use_memory = ['chat', 'action', 'embedding'],
    return_intermediate_steps=True, #TODO: this variable has to be true for the code to work, we should handle this properly instead of a mandatory boolean value
    verbose=True
)

/home/parikh92/sciborg/ai/agents/core2.py:121: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memories.append(ConversationBufferWindowMemory(


In [5]:
new_agent.invoke(
    {
        "input": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
)



> Entering new AgentExecutor chain...
Thought: To find the Ki of pzm21 to the Mu opioid receptor, I first need to find the compound ID (CID) of pzm21. I will use the get_cid_from_name function for this.

Action:
```
{
    "action": "get_cid_from_name",
    "action_input": {
        "inp": "pzm21"
    }
}
```{'response': {'', '121596705', '124186394'}}The compound ID for pzm21 is either '121596705' or '124186394'. Now, I will use the get_assay_results_for_compund function to get the assay results for this compound. 

Action:
```
{
    "action": "get_assay_results_for_compund",
    "action_input": {
        "cid": "121596705",
        "activity_name": "Ki"
    }
}
```{'AID': {1: '1573390', 5: '1609954', 17: '1806958', 18: '1806958', 19: '1806958', 20: '1806958'}, 'Panel Member ID': {1: '', 5: '', 17: '', 18: '', 19: '', 20: ''}, 'SID': {1: '440143506', 5: '440143506', 17: '440726618', 18: '440726618', 19: '440726618', 20: '440726618'}, 'CID': {1: '121596705', 5: '121596705', 17: '12159

/home/parikh92/sciborg/ai/memory/internal_logging.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt) #this will be deprecated soon



> Finished chain.


{'input': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': '',
 'past_action_log': '',
 'output': "The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.",
 'intermediate_steps': [(AgentAction(tool='get_cid_from_name', tool_input={'inp': 'pzm21'}, log='Thought: To find the Ki of pzm21 to the Mu opioid receptor, I first need to find the compound ID (CID) of pzm21. I will use the get_cid_from_name function for this.\n\nAction:\n```\n{\n    "action": "get_cid_from_name",\n    "action_input": {\n        "inp": "pzm21"\n    }\n}\n```'),
   "{'response': {'', '121596705', '124186394'}}"),
  (AgentAction(tool='get_assay_results_for_compund', tool_input={'cid': '121596705', 'activity

In [6]:
new_agent.memory.memories
#TODO: how to find index of memory with specific type in python (find index of an element in a list)


[ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?', additional_kwargs={}, response_metadata={}), AIMessage(content="The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.", additional_kwargs={}, response_metadata={})]), output_key='output', input_key='input', memory_key='chat_history'),
 CustomActionLogSummaryMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1499a45d4790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1499a4989e50>, root_client=<openai.OpenAI object at 0x1499a45defd0>, root_async_client=<openai.AsyncOpenAI obje

In [7]:
new_agent.invoke(
    {
        "input": "What is the molecular weight of benzene?"
    }
)



> Entering new AgentExecutor chain...
Thought: The molecular weight of a substance is one of its basic properties. Using the PubChem API, we can fetch these properties for a given compound.

Action:
```
{
    "action": "get_compound_property_table",
    "action_input": {
        "inp": "benzene",
        "inp_format": "name",
        "inp_type": "compound",
        "property_list": "MolecularWeight"
    }
}
```MolecularWeight
{'PropertyTable': {'Properties': [{'CID': 241, 'MolecularWeight': '78.11'}]}}The molecular weight of benzene is 78.11 g/mol as per the information retrieved from PubChem.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The molecular weight of benzene is 78.11 g/mol."
}
```

> Finished chain.


{'input': 'What is the molecular weight of benzene?',
 'chat_history': "Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?\nAI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.",
 'past_action_log': 'I have found the compound ID (CID) of pzm21, which is either \'121596705\' or \'124186394\'. Using the CID \'121596705\', I have retrieved the assay results for this compound. The results show that pzm21 is active in several assays. The activity values (Ki) range from 0.0011 uM to 1 uM. The assays include displacement of [3H]-Diprenorphine from the mu opioid receptor expressed in sf9 insect cell membranes, inhibition of the mu opioid receptor assessed as reduction in intracellular cAMP ac

In [8]:
type(new_agent.memory.memories)

list

In [9]:
new_agent.save_memory('core_memory.json')

Memory file name provided, saving memory to core_memory.json
Memory saved successfully
Memory file saved at core_memory.json


In [8]:
agent_with_context.invoke(
    {
        "input": "What all do you recall from the previous conversation?"
    }
)



> Entering new AgentExecutor chain...
Action:
```
{
    "action": "Final Answer",
    "action_input": "From the previous conversation, I recall that the compound pzm21 is active in several assays related to the Mu opioid receptor. It has activity values (Ki) ranging from 0.0011 uM to 1 uM. The assays include displacement of [3H]-Diprenorphine from the mu opioid receptor expressed in sf9 insect cell membranes, inhibition of the mu opioid receptor assessed as reduction in intracellular cAMP accumulation, and several radioligand binding assays from US Patent US11484525 titled 'Mu opioid receptor modulators'. The assays are all confirmatory. The PubMed IDs associated with some of the assays are '29939744' and '31550662'. Additionally, the molecular weight of benzene was identified to be 78.11 g/mol."
}
```


/home/parikh92/sciborg/ai/memory/internal_logging.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt) #this will be deprecated soon



> Finished chain.


{'input': 'What all do you recall from the previous conversation?',
 'chat_history': "Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?\nAI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.\nHuman: What is the molecular weight of benzene?\nAI: The molecular weight of benzene is 78.11 g/mol.",
 'past_action_log': 'I have found the compound ID (CID) of pzm21, which is either \'121596705\' or \'124186394\'. Using the CID \'121596705\', I have retrieved the assay results for this compound. The results show that pzm21 is active in several assays. The activity values (Ki) range from 0.0011 uM to 1 uM. The assays include displacement of [3H]-Diprenorphine from the mu opioid receptor express

In [8]:
agent_with_context.invoke(
    {
        "input": "Find the molecular weight of benzene? or do you know it already?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human user is asking for the molecular weight of benzene. I know the molecular weight of benzene is 78.11 g/mol from the previous conversation. However, to ensure the accuracy and source traceability, I will use the PubChem API function 'get_compound_property_table' to fetch the information from the PubChem database.

Action:
```
{
    "action": "get_compound_property_table",
    "action_input": {
        "inp": "benzene",
        "inp_format": "name",
        "inp_type": "compound",
        "property_list": "MolecularWeight"
    }
}
```MolecularWeight
{'PropertyTable': {'Properties': [{'CID': 241, 'MolecularWeight': '78.11'}]}}The PubChem database confirms that the molecular weight of benzene is 78.11 g/mol. Now, I can provide this information to the user.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The molecular weight of benzene is 78.11 g/mol according to the PubChem database."
}
```

> Finished chain.


{'input': 'Find the molecular weight of benzene? or do you know it already?',
 'chat_history': "Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?\nAI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.\nHuman: What is the molecular weight of benzene?\nAI: The molecular weight of benzene is 78.11 g/mol.\nHuman: What all do you recall from the previous conversation?\nAI: From the previous conversation, I recall that the Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This info

In [9]:
agent_with_context.invoke(
    {
        "input": "Find the molecular weight of benzene if you don't know it already?"
    }
)



> Entering new AgentExecutor chain...
Thought: The molecular weight of benzene has been queried again. As this information was already obtained in the previous conversation and confirmed, I can directly provide the answer without using a tool.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The molecular weight of benzene is 78.11 g/mol according to the PubChem database."
}
```

> Finished chain.


{'input': "Find the molecular weight of benzene if you don't know it already?",
 'chat_history': "Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?\nAI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.\nHuman: What is the molecular weight of benzene?\nAI: The molecular weight of benzene is 78.11 g/mol.\nHuman: What all do you recall from the previous conversation?\nAI: From the previous conversation, I recall that the Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This in

In [7]:
agent_with_context.invoke(
    {
        "input": "Find the molecular weight of benzene?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking for the molecular weight of benzene. I can find this information using the 'get_compound_property_table' function and specifying 'MolecularWeight' as the property to look for.

Action:
```
{
  "action": "get_compound_property_table",
  "action_input": {
    "inp": "benzene",
    "inp_format": "name",
    "inp_type": "compound",
    "property_list": "MolecularWeight"
  }
}
```MolecularWeight
{'PropertyTable': {'Properties': [{'CID': 241, 'MolecularWeight': '78.11'}]}}The molecular weight of benzene is 78.11 g/mol according to the PubChem database.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The molecular weight of benzene is 78.11 g/mol according to the PubChem database."
}
```

> Finished chain.


{'input': 'Find the molecular weight of benzene?',
 'chat_history': "Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?\nAI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.\nHuman: What is the molecular weight of benzene?\nAI: The molecular weight of benzene is 78.11 g/mol.\nHuman: What all do you recall from the previous conversation?\nAI: From the previous conversation, I recall that the Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from Pub

In [13]:
agent_with_context.memory.memories[0].chat_memory.messages

[HumanMessage(content='What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM according to the assay 'Displacement of [3H]-Diprenorphine from mu opioid receptor (unknown origin) expressed in sf9 insect cell membranes after 1 hr by liquid scintillation counting method' (Assay ID: 1573390). This information is sourced from PubChem.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the molecular weight of benzene?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The molecular weight of benzene is 78.11 g/mol.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What all do you recall from the previous conversation?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="From the previous conversation, I recall that we discussed the Ki of pzm21 to the Mu opioid 

In [ ]:
agent_with_context.invoke(
    {
        "input": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking for the Ki of pzm21 to the Mu opioid receptor and wants a specific assay cited in the response. I will use the tool `get_assay_results_for_compund` to get the assay results for the compound pzm21.

Action:
```
{
    "action": "get_assay_results_for_compund",
    "action_input": {
        "cid": "121596705",
        "activity_name": "Ki"
    }
}
```{'AID': {1: '1573390', 5: '1609954', 17: '1806958', 18: '1806958', 19: '1806958', 20: '1806958'}, 'Panel Member ID': {1: '', 5: '', 17: '', 18: '', 19: '', 20: ''}, 'SID': {1: '440143506', 5: '440143506', 17: '440726618', 18: '440726618', 19: '440726618', 20: '440726618'}, 'CID': {1: '121596705', 5: '121596705', 17: '121596705', 18: '121596705', 19: '121596705', 20: '121596705'}, 'Activity Outcome': {1: 'Active', 5: 'Active', 17: 'Active', 18: 'Active', 19: 'Active', 20: 'Active'}, 'Target GI': {1: '2851402', 5: '2851402', 17: '2851402', 18: '116242691', 19: '116242691', 20:

{'input': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': 'Human: Find the molecular weight of benzene?\nAI: The molecular weight of benzene is 78.11.',
 'past_action_log': 'I have found the compound ID (CID) of pzm21, which is either \'121596705\' or \'124186394\'. Using the CID \'121596705\', I have retrieved the assay results for this compound. The results show that pzm21 is active in several assays. The activity values (Ki) range from 0.0011 uM to 1 uM. The assays include displacement of [3H]-Diprenorphine from the mu opioid receptor expressed in sf9 insect cell membranes, inhibition of the mu opioid receptor assessed as reduction in intracellular cAMP accumulation, and several radioligand binding assays from US Patent US11484525 titled "Mu opioid receptor modulators". The assays are all confirmatory. The PubMed IDs associated with some of the assays are \'29939744\' and \'31550662\'. I have also found the molecular wei

In [ ]:
print(

SyntaxError: incomplete input (149104261.py, line 1)

In [ ]:
pubchem_driver_based_agent = create_linqx_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    verbose=True
)

NameError: name 'pubchem_command_microservice' is not defined

Processing functions:  12%|█▎        | 1/8 [00:01<00:13,  1.93s/function]

{'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '2e6b29df-25eb-4e50-bd4b-5c66bae91dcf', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}
<class 'dict'>


Processing functions:  25%|██▌       | 2/8 [00:03<00:10,  1.76s/function]

{'name': 'close_lid', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Closes the lid on the microwave synthesizer. Must be run prior to running heating.', 'uuid': '2e6b29df-25eb-4e50-bd4b-5c66bae91dcf', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'is_optional': False, 'is_list': False, 'desc': 'the id of the current session'}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions:  38%|███▊      | 3/8 [00:05<00:08,  1.66s/function]

...

{'name': 'open_lid', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Opens the lid on the microwave synthesizer. Must be run prior to loading a vial.', 'uuid': '2e6b29df-25eb-4e50-bd4b-5c66bae91dcf', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'is_optional': False, 'is_list': False, 'desc': 'the id of the current session'}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions:  88%|████████▊ | 7/8 [00:12<00:01,  1.81s/function]

{'name': 'unload_vial', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Unloads a vial from the microwave synthesizer. Must be run after heating.', 'uuid': '2e6b29df-25eb-4e50-bd4b-5c66bae91dcf', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'is_optional': False, 'is_list': False, 'desc': 'the id of the current session'}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>


Processing functions: 100%|██████████| 8/8 [00:16<00:00,  2.08s/function]

{'name': 'update_heating_parameters', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Sets the heating parameters of the microwave synthesizer. Must be run prior to heating.', 'uuid': '2e6b29df-25eb-4e50-bd4b-5c66bae91dcf', 'parameters': {'duration': {'name': 'duration', 'data_type': 'int', 'precision': -1, 'upper_limit': 60, 'lower_limit': 1, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A duration in minutes for the heating process'}, 'temperature': {'name': 'temperature', 'data_type': 'int', 'precision': -1, 'upper_limit': 250, 'lower_limit': 25, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A temperature in celsius for the heating process'}, 'pressure': {'name': 'pressure', 'data_type': 'float', 'precision': -1, 'upper_limit': 10, 'lower_limit': 1, 'is_optional': False, 'is_list': False, 'default': None, 'from_var': False, 'desc': 'A pressure in atm for the heating process'}, 'session_ID': {'name': 'sess

In [ ]:
test_agent = create_linqx_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    use_memory = 'both',
    verbose=True
)

test_agent.invoke(
    {
        "input": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
)

/home/parikh92/sciborg/ai/agents/core.py:194: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationBufferWindowMemory(memory_key='chat_history', input_key='input', output_key='output')




> Entering new AgentExecutor chain...
Thought: To get the Ki of PZM21 to the Mu opioid receptor, I first need to get the CID of PZM21. 

Action:
```
{
    "action": "get_cid_from_name",
    "action_input": {
        "inp": "pzm21"
    }
}
```{'response': {'', '124186394', '121596705'}}The CID for PZM21 is either 124186394 or 121596705. Next, I will use this CID to get the assay results for this compound where the activity name is Ki.

Action:
```
{
    "action": "get_assay_results_for_compund",
    "action_input": {
        "cid": "124186394",
        "activity_name": "Ki"
    }
}
```{'AID': {0: '1806958', 1: '1806958', 2: '1806958', 3: '1806958', 4: '1806958', 5: '1806958'}, 'Panel Member ID': {0: '', 1: '', 2: '', 3: '', 4: '', 5: ''}, 'SID': {0: '475725385', 1: '475725385', 2: '475725385', 3: '475725385', 4: '475725385', 5: '475725385'}, 'CID': {0: '124186394', 1: '124186394', 2: '124186394', 3: '124186394', 4: '124186394', 5: '124186394'}, 'Activity Outcome': {0: 'Active', 1: 'Ac

/home/parikh92/sciborg/ai/memory/internal_logging.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt)



> Finished chain.


{'input': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': '',
 'past_action_log': '',
 'output': "The Ki of PZM21 to the Mu opioid receptor is available in the assay with the AID 1806958. The values range from 0.042 uM to 4.1 uM. This data comes from the Radioligand Binding Assay from US Patent US11484525: 'Mu opioid receptor modulators'.",
 'intermediate_steps': [(AgentAction(tool='get_cid_from_name', tool_input={'inp': 'pzm21'}, log='Thought: To get the Ki of PZM21 to the Mu opioid receptor, I first need to get the CID of PZM21. \n\nAction:\n```\n{\n    "action": "get_cid_from_name",\n    "action_input": {\n        "inp": "pzm21"\n    }\n}\n```'),
   "{'response': {'', '124186394', '121596705'}}"),
  (AgentAction(tool='get_assay_results_for_compund', tool_input={'cid': '124186394', 'activity_name': 'Ki'}, log='The CID for PZM21 is either 124186394 or 121596705. Next, I will use this CID to get the assay results for this c

In [ ]:
test_agent.__dir__()

['name',
 'memory',
 'callbacks',
 'verbose',
 'tags',
 'metadata',
 'callback_manager',
 'agent',
 'tools',
 'return_intermediate_steps',
 'max_iterations',
 'max_execution_time',
 'early_stopping_method',
 'handle_parsing_errors',
 'trim_intermediate_steps',
 '__module__',
 '__annotations__',
 '__doc__',
 'from_agent_and_tools',
 'validate_tools',
 'validate_runnable_agent',
 '_action_agent',
 'save',
 'save_agent',
 'iter',
 'input_keys',
 'output_keys',
 'lookup_tool',
 '_should_continue',
 '_return',
 '_areturn',
 '_consume_next_step',
 '_take_next_step',
 '_iter_next_step',
 '_perform_agent_action',
 '_atake_next_step',
 '_aiter_next_step',
 '_aperform_agent_action',
 '_call',
 '_acall',
 '_get_tool_return',
 '_prepare_intermediate_steps',
 'stream',
 'astream',
 'model_config',
 '__class_vars__',
 '__private_attributes__',
 '__parameters__',
 '__abstractmethods__',
 '_abc_impl',
 '__pydantic_custom_init__',
 '__pydantic_post_init__',
 '__pydantic_decorators__',
 '__pydantic_gene

In [ ]:
type(test_agent.memory)

langchain.memory.combined.CombinedMemory

In [ ]:
print(test_agent.memory.memories[0].buffer)

Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?
AI: The Ki of PZM21 to the Mu opioid receptor is available in the assay with the AID 1806958. The values range from 0.042 uM to 4.1 uM. This data comes from the Radioligand Binding Assay from US Patent US11484525: 'Mu opioid receptor modulators'.


In [ ]:
test_agent.memory.memories[1].__dir__()

['human_prefix',
 'ai_prefix',
 'llm',
 'prompt',
 'summary_message_cls',
 'chat_memory',
 'output_key',
 'input_key',
 'return_messages',
 'buffer',
 'memory_key',
 'filtered_tool_list',
 '__module__',
 '__annotations__',
 '__doc__',
 'save_context',
 'model_config',
 '__class_vars__',
 '__private_attributes__',
 '__abstractmethods__',
 '_abc_impl',
 '__pydantic_custom_init__',
 '__pydantic_post_init__',
 '__pydantic_decorators__',
 '__pydantic_generic_metadata__',
 '__pydantic_complete__',
 '__pydantic_parent_namespace__',
 '__pydantic_fields__',
 '__pydantic_core_schema__',
 '__pydantic_validator__',
 '__pydantic_serializer__',
 '__signature__',
 '__pydantic_computed_fields__',
 'from_messages',
 'memory_variables',
 'load_memory_variables',
 'validate_prompt_input_variables',
 'clear',
 '__init__',
 '_get_input_output',
 'asave_context',
 'aclear',
 'aload_memory_variables',
 'is_lc_serializable',
 'get_lc_namespace',
 'lc_secrets',
 'lc_attributes',
 'lc_id',
 '__repr_args__',
 't

In [ ]:
test_agent.memory.memories[1].model_fields

{'human_prefix': FieldInfo(annotation=str, required=False, default='Human'),
 'ai_prefix': FieldInfo(annotation=str, required=False, default='AI'),
 'llm': FieldInfo(annotation=BaseLanguageModel, required=True),
 'prompt': FieldInfo(annotation=BasePromptTemplate, required=False, default=PromptTemplate(input_variables=['new_lines', 'summary'], input_types={}, partial_variables={}, template='\nYou will receive a list of python dictionaries representing an AI agents internal though/action/observation process.\n\nThe first element of each tuple is the agent action, the second element is the observation of that action.\n\nYou should summarize the action and observation that the AI agent had for each element of this list.\nYou MUST include all values of each action\'s observation in your summary as they may be relevent later.\n\nFor example the following list of one agent thought/action/observation\n\nCurrent Summary:\nI have petted the cow and the cow says moo.\n\nHuman: Can you pet the dog

In [ ]:
test_agent.memory.memories[1].dict()

/tmp/ipykernel_2005794/4175375629.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  test_agent.memory.memories[1].dict()


{'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'llm': {'name': None},
 'prompt': {'name': None,
  'input_variables': ['new_lines', 'summary'],
  'optional_variables': [],
  'output_parser': None,
  'partial_variables': {},
  'metadata': None,
  'tags': None},
 'summary_message_cls': langchain_core.messages.system.SystemMessage,
 'chat_memory': {'messages': [{'content': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': None},
   {'content': '[(AgentAction(tool=\'get_cid_from_name\', tool_input={\'inp\': \'pzm21\'}, log=\'Thought: To get the Ki of PZM21 to the Mu opioid receptor, I first need to get the CID of PZM21. \\n\\nAction:\\n```\\n{\\n    "action": "get_cid_from_name",\\n    "action_input": {\\n        "inp": "pzm21"\\n    }\\n}\\n```\'), "{\'response\': {\'\', \'124186394\', \'121596705\'}}"), (AgentAction(tool=\'get_assay_results

In [ ]:
import json
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import HumanMessage, AIMessage

# Function to save memory
def save_memory(memory, file_path):
    data = {
        "messages": [
            {"type": type(msg).__name__, "content": msg.content}
            for msg in memory.chat_memory.messages
        ]
    }
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)

# Function to load memory
def load_memory(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    messages = []
    for msg in data["messages"]:
        if msg["type"] == "HumanMessage":
            messages.append(HumanMessage(content=msg["content"]))
        elif msg["type"] == "AIMessage":
            messages.append(AIMessage(content=msg["content"]))

    # Reconstruct the memory object
    memory = ConversationBufferWindowMemory()
    memory.chat_memory.messages = messages
    return memory

# Example usage:
memory = test_agent.memory.memories[0]

# Save the memory
save_memory(memory, "memory.json")

# Load the memory
loaded_memory = load_memory("memory.json")

# Verify loaded memory
print(loaded_memory.chat_memory.messages)
print(type(loaded_memory.chat_memory.messages))

[HumanMessage(content='What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?', additional_kwargs={}, response_metadata={}), AIMessage(content="The Ki of PZM21 to the Mu opioid receptor is available in the assay with the AID 1806958. The values range from 0.042 uM to 4.1 uM. This data comes from the Radioligand Binding Assay from US Patent US11484525: 'Mu opioid receptor modulators'.", additional_kwargs={}, response_metadata={})]
<class 'list'>


In [ ]:
rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = 'str(project_root / 'embeddings/NIH_docs_embeddings',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    use_memory = 'both',
    agent_as_a_tool=False,
    verbose=True
)


In [ ]:
rag_agent.memory.memories[2].model_dump()

{'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'llm': {'name': None},
 'prompt': {'name': None,
  'input_variables': ['new_lines', 'summary'],
  'optional_variables': [],
  'output_parser': None,
  'partial_variables': {},
  'metadata': None,
  'tags': None},
 'summary_message_cls': langchain_core.messages.system.SystemMessage,
 'chat_memory': {'messages': []},
 'output_key': 'intermediate_steps',
 'input_key': 'input',
 'return_messages': False,
 'buffer': '',
 'memory_key': 'rag_log',
 'filtered_tool_list': ['call_provided_Agent', 'call_RAG_agent']}

In [ ]:
agent2 = create_linqx_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    use_memory = 'both',
    verbose=True
)

In [ ]:
agent2.memory.memories[0].dict()

/tmp/ipykernel_2005794/541317784.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  agent2.memory.memories[0].dict()


{'chat_memory': {'messages': []},
 'output_key': 'output',
 'input_key': 'input',
 'return_messages': False,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'chat_history',
 'k': 5}

In [ ]:
agent2.memory.memories[1].model_dump().keys()

dict_keys(['human_prefix', 'ai_prefix', 'llm', 'prompt', 'summary_message_cls', 'chat_memory', 'output_key', 'input_key', 'return_messages', 'buffer', 'memory_key', 'filtered_tool_list'])

In [ ]:
test_agent.memory.memories[1].model_dump().keys()

dict_keys(['human_prefix', 'ai_prefix', 'llm', 'prompt', 'summary_message_cls', 'chat_memory', 'output_key', 'input_key', 'return_messages', 'buffer', 'memory_key', 'filtered_tool_list'])

In [ ]:
agent2.memory.memories[1].model_dump()

{'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'llm': {'name': None},
 'prompt': {'name': None,
  'input_variables': ['new_lines', 'summary'],
  'optional_variables': [],
  'output_parser': None,
  'partial_variables': {},
  'metadata': None,
  'tags': None},
 'summary_message_cls': langchain_core.messages.system.SystemMessage,
 'chat_memory': {'messages': []},
 'output_key': 'intermediate_steps',
 'input_key': 'input',
 'return_messages': False,
 'buffer': '',
 'memory_key': 'past_action_log',
 'filtered_tool_list': ['get_assay_description_from_AID',
  'get_assay_name_from_aid',
  'get_assay_results_for_compund',
  'get_cid_from_name',
  'get_compound_property_table',
  'get_description',
  'get_smiles_from_name',
  'get_synonym']}

In [ ]:
test_agent.memory.memories[1].model_dump()

{'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'llm': {'name': None},
 'prompt': {'name': None,
  'input_variables': ['new_lines', 'summary'],
  'optional_variables': [],
  'output_parser': None,
  'partial_variables': {},
  'metadata': None,
  'tags': None},
 'summary_message_cls': langchain_core.messages.system.SystemMessage,
 'chat_memory': {'messages': [{'content': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': None},
   {'content': '[(AgentAction(tool=\'get_cid_from_name\', tool_input={\'inp\': \'pzm21\'}, log=\'Thought: To get the Ki of PZM21 to the Mu opioid receptor, I first need to get the CID of PZM21. \\n\\nAction:\\n```\\n{\\n    "action": "get_cid_from_name",\\n    "action_input": {\\n        "inp": "pzm21"\\n    }\\n}\\n```\'), "{\'response\': {\'\', \'124186394\', \'121596705\'}}"), (AgentAction(tool=\'get_assay_results

In [ ]:
agent2.memory.memories[0].model_dump()

{'chat_memory': {'messages': []},
 'output_key': 'output',
 'input_key': 'input',
 'return_messages': False,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'chat_history',
 'k': 5}

In [ ]:
test_agent.memory.memories[0].model_dump()

{'chat_memory': {'messages': [{'content': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': None},
   {'content': "The Ki of PZM21 to the Mu opioid receptor is available in the assay with the AID 1806958. The values range from 0.042 uM to 4.1 uM. This data comes from the Radioligand Binding Assay from US Patent US11484525: 'Mu opioid receptor modulators'.",
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'ai',
    'name': None,
    'id': None}]},
 'output_key': 'output',
 'input_key': 'input',
 'return_messages': False,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'chat_history',
 'k': 5}

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class MicrowaveSynthesizerFSA(BaseModel):
    sessionID: str | None = Field(defaullt=None, description='ID of the session allocation or None if no session allocated. Changed by allocating the session')
    lid_status: Literal['open', 'closed'] = Field(default='closed', description='status of the lid. Changed by opening or closing the lid')
    vial_status: Literal['loaded', 'unloaded'] = Field(default='unloaded', description='status of the vial. Changed by loading/unloading the vial')
    vial: str | None = Field(default=None, description='Identifier of the vial loaded, None if no vial is loaded. Changed by loading/unloading the vial')
    heating_status: Literal['not_heating', 'heating'] = Field(default='not_heating', description='Status of heating. Only changed by heating the vial, not by updating the parameters')
    temp: int | None = Field(default=None, description='set temperature to heat at, None if not currently set. Changed by updating the heating parameters')
    duration: int | None = Field(default=None, description='set duration to heat for, None if not currently set. Changed by updating the heating parameters')
    pressure: float | None = Field(default=None, description='set pressure to heat at, None if not currently set. Changed by updating the heating parameters')

ms_agent_chat_fsa_memory = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    llm=ChatOpenAI(model='gpt-4', temperature=0),
    use_memory='both',
    human_interaction=True,
    verbose=True,
    agent_as_a_fsa=True,
    fsa_object=MicrowaveSynthesizerFSA
)

KeyError: 'prompt_template'